In [ ]:
import mnist
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

In [ ]:
x_train = mnist.train_images()
y_train = mnist.train_labels()
x_test = mnist.test_images()
y_test = mnist.test_labels()

In [ ]:
def get_features(image):
    contours, _ = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]

    features = {}

    area = cv.contourArea(cnt)
    features['area'] = area
    perimeter = cv.arcLength(cnt, True)
    features['perimeter'] = perimeter
    
    x, y, w, h = cv.boundingRect(cnt)
    features['aspect_ratio'] = float(w) / float(h)
    
    rect_area = w*h
    features['extent'] = float(area) / float(rect_area)
    
    hull = cv.convexHull(cnt)
    hull_area = cv.contourArea(hull)
    features['hull_area'] = hull_area
    try:
        features['solidity'] = float(area) / float(hull_area)
    except:
        features['solidity'] = 0.0

    try:
        (x, y), (MA, ma), angle = cv.fitEllipse(cnt)
    except:
        angle, MA, ma = 0.0, 0.0, 0.0
    features['angle'] = angle
    features['major_axis_length'] = MA
    features['minor_axis_length'] = ma
    return features

In [ ]:
def convert(samples):
    images = []
    for i, image in enumerate(samples):
        if i % 1000 == 0:
            print(i)
        
        features = get_features(image)
        features = np.array([v[1] for v in sorted(features.items(), key=lambda x: x[0])])
        images.append(features)

    return np.stack(images)

np.save('mnist_features_train.npy', convert(x_train))
np.save('mnist_features_test.npy', convert(x_test))